<a href="https://colab.research.google.com/github/alantang1369/Household_Power_7_Days_Prediction/blob/main/Household_Power_7_Days_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib
import zipfile

import pandas as pd
import tensorflow as tf


In [ ]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip'
urllib.request.urlretrieve(url, 'household_power.zip')
with zipfile.ZipFile('household_power.zip', 'r') as zip_ref:
        zip_ref.extractall()

In [ ]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data


In [ ]:
# This function is used to map the time series dataset into windows of
# features and respective targets, to prepare it for training and
# validation. First element of the first window will be the first element of
# the dataset. Consecutive windows are constructed by shifting
# the starting position of the first window forward, one at a time (indicated
# by shift=1). For a window of n_past number of observations of all the time
# indexed variables in the dataset, the target for the window
# is the next n_future number of observations of these variables, after the
# end of the window.


def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)



# This function loads the data from CSV file, normalizes the data and
# splits the dataset into train and validation data. It also uses
# windowed_dataset() to split the data into windows of observations and
# targets. Finally it defines, compiles and trains a neural network. This
# function returns the final trained model.

#download_and_extract_data()
    # Reads the dataset from the CSV.
df = pd.read_csv('household_power_consumption.csv', sep=',',
                  infer_datetime_format=True, index_col='datetime', header=0)

# Number of features in the dataset. We use all features as predictors to
# predict all features at future time steps.
N_FEATURES = len(df.columns)

# Normalizes the data
data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

# Splits the data into training and validation sets.
SPLIT_TIME = int(len(data) * 0.5)
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]


tf.keras.backend.clear_session()
tf.random.set_seed(42)


BATCH_SIZE = 32

# Number of past time steps based on which future observations should be
# predicted
N_PAST = 24

# Number of future time steps which are to be predicted.
N_FUTURE = 24
# By how many positions the window slides to create a new window
# of observations.
SHIFT = 1

# Code to create windowed train and validation datasets.
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                              n_past=N_PAST, n_future=N_FUTURE,
                              shift=SHIFT)
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)



<ipython-input-4-9647fc39e808>:29: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('household_power_consumption.csv', sep=',',


In [ ]:
df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [ ]:
output_shape = (N_FUTURE, N_FEATURES)
output_shape

(24, 7)

In [ ]:
# Code to define your model.
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(N_PAST, N_FEATURES)),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(128 , activation= 'relu'),
    tf.keras.layers.Dense(N_FEATURES)
])

# Code to train and compile the model
optimizer = tf.keras.optimizers.Adam()
model.compile(
    loss="mae",
    optimizer=optimizer,
    metrics=["mae"]
)


In [ ]:
model.fit(
    train_set,
    epochs=100,
    verbose=1, # only print 1 line per epoch
    validation_data=valid_set,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

Epoch 1/100
1349/1349 [==============================] - 11s 7ms/step - loss: 0.0656 - mae: 0.0656 - val_loss: 0.0594 - val_mae: 0.0594
Epoch 2/100
1349/1349 [==============================] - 9s 7ms/step - loss: 0.0580 - mae: 0.0580 - val_loss: 0.0566 - val_mae: 0.0566
Epoch 3/100
1349/1349 [==============================] - 9s 7ms/step - loss: 0.0576 - mae: 0.0576 - val_loss: 0.0576 - val_mae: 0.0576
Epoch 4/100
1349/1349 [==============================] - 9s 7ms/step - loss: 0.0573 - mae: 0.0573 - val_loss: 0.0567 - val_mae: 0.0567
Epoch 5/100
1349/1349 [==============================] - 9s 7ms/step - loss: 0.0567 - mae: 0.0567 - val_loss: 0.0570 - val_mae: 0.0570
Epoch 6/100
1349/1349 [==============================] - 9s 7ms/step - loss: 0.0565 - mae: 0.0565 - val_loss: 0.0566 - val_mae: 0.0566
Epoch 7/100
1349/1349 [==============================] - 9s 7ms/step - loss: 0.0562 - mae: 0.0562 - val_loss: 0.0550 - val_mae: 0.0550
Epoch 8/100
1349/1349 [==============================]

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 24, 128)           1024      
                                                                 
 dense_2 (Dense)             (None, 24, 7)             903       
                                                                 
Total params: 1927 (7.53 KB)
Trainable params: 1927 (7.53 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
